In [ ]:
pip install aiocron 

In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup



In [2]:

import os
from typing import Any, AsyncGenerator, Awaitable, Callable, Dict, Optional
from aiocron import crontab
from pydantic import BaseModel, Field
from pytz import BaseTzInfo

from metagpt.actions.action import Action
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message

# fix SubscriptionRunner not fully defined
from metagpt.environment import Environment as _  # noqa: F401

2025-01-23 20:21:41.246 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to e:\wow-agent\notebook


In [3]:

class SubscriptionRunner(BaseModel):
    """A simple wrapper to manage subscription tasks for different roles using asyncio.
    Example:
        >>> import asyncio
        >>> from metagpt.subscription import SubscriptionRunner
        >>> from metagpt.roles import Searcher
        >>> from metagpt.schema import Message
        >>> async def trigger():
        ...     while True:
        ...         yield Message("the latest news about OpenAI")
        ...         await asyncio.sleep(3600 * 24)
        >>> async def callback(msg: Message):
        ...     print(msg.content)
        >>> async def main():
        ...     pb = SubscriptionRunner()
        ...     await pb.subscribe(Searcher(), trigger(), callback)
        ...     await pb.run()
        >>> asyncio.run(main())
    """

    tasks: Dict[Role, asyncio.Task] = Field(default_factory=dict)

    class Config:
        arbitrary_types_allowed = True

    async def subscribe(
        self,
        role: Role,
        trigger: AsyncGenerator[Message, None],
        callback: Callable[
            [
                Message,
            ],
            Awaitable[None],
        ],
    ):
        """Subscribes a role to a trigger and sets up a callback to be called with the role's response.
        Args:
            role: The role to subscribe.
            trigger: An asynchronous generator that yields Messages to be processed by the role.
            callback: An asynchronous function to be called with the response from the role.
        """
        loop = asyncio.get_running_loop()

        async def _start_role():
            async for msg in trigger:
                resp = await role.run(msg)
                await callback(resp)

        self.tasks[role] = loop.create_task(_start_role(), name=f"Subscription-{role}")

    async def unsubscribe(self, role: Role):
        """Unsubscribes a role from its trigger and cancels the associated task.
        Args:
            role: The role to unsubscribe.
        """
        task = self.tasks.pop(role)
        task.cancel()

    async def run(self, raise_exception: bool = True):
        """Runs all subscribed tasks and handles their completion or exception.
        Args:
            raise_exception: _description_. Defaults to True.
        Raises:
            task.exception: _description_
        """
        i=0
        while True:
            i+=1
            for role, task in self.tasks.items():
                i=0
                if task.done():
                    if task.exception():
                        if raise_exception:
                            raise task.exception()
                        logger.opt(exception=task.exception()).error(
                            f"Task {task.get_name()} run error"
                        )
                    else:
                        logger.warning(
                            f"Task {task.get_name()} has completed. "
                            "If this is unexpected behavior, please check the trigger function."
                        )
                    self.tasks.pop(role)
                    break
            else:
                await asyncio.sleep(1)
            if i>0:
                break

In [4]:

TRENDING_ANALYSIS_PROMPT = """# Requirements
You are a GitHub Trending Analyst, aiming to provide users with insightful and personalized recommendations based on the latest
GitHub Trends. Based on the context, fill in the following missing information, generate engaging and informative titles, 
ensuring users discover repositories aligned with their interests.

# The title about Today's GitHub Trending
## Today's Trends: Uncover the Hottest GitHub Projects Today! Explore the trending programming languages and discover key domains capturing developers' attention. From ** to **, witness the top projects like never before.
## The Trends Categories: Dive into Today's GitHub Trending Domains! Explore featured projects in domains such as ** and **. Get a quick overview of each project, including programming languages, stars, and more.
## Highlights of the List: Spotlight noteworthy projects on GitHub Trending, including new tools, innovative projects, and rapidly gaining popularity, focusing on delivering distinctive and attention-grabbing content for users.
---
# Format Example


# [Title]

## Today's Trends
Today, ** and ** continue to dominate as the most popular programming languages. Key areas of interest include **, ** and **.
The top popular projects are Project1 and Project2.

## The Trends Categories
1. Generative AI
    - [Project1](https://github/xx/project1): [detail of the project, such as star total and today, language, ...]
    - [Project2](https://github/xx/project2): ...
...

## Highlights of the List
1. [Project1](https://github/xx/project1): [provide specific reasons why this project is recommended].
...

---
# Github Trending
{trending}
"""


class CrawlOSSTrending(Action):
    async def run(self, url: str = "https://github.com/trending"):
        async with aiohttp.ClientSession() as client:
            async with client.get(url) as response:
                response.raise_for_status()
                html = await response.text()

        soup = BeautifulSoup(html, "html.parser")

        repositories = []

        for article in soup.select("article.Box-row"):
            repo_info = {}

            repo_info["name"] = (
                article.select_one("h2 a")
                .text.strip()
                .replace("\n", "")
                .replace(" ", "")
            )
            repo_info["url"] = (
                "https://github.com" + article.select_one("h2 a")["href"].strip()
            )

            # Description
            description_element = article.select_one("p")
            repo_info["description"] = (
                description_element.text.strip() if description_element else None
            )

            # Language
            language_element = article.select_one(
                'span[itemprop="programmingLanguage"]'
            )
            repo_info["language"] = (
                language_element.text.strip() if language_element else None
            )

            # Stars and Forks
            stars_element = article.select("a.Link--muted")[0]
            forks_element = article.select("a.Link--muted")[1]
            repo_info["stars"] = stars_element.text.strip()
            repo_info["forks"] = forks_element.text.strip()

            # Today's Stars
            today_stars_element = article.select_one(
                "span.d-inline-block.float-sm-right"
            )
            repo_info["today_stars"] = (
                today_stars_element.text.strip() if today_stars_element else None
            )

            repositories.append(repo_info)

        return repositories


class AnalysisOSSTrending(Action):
    async def run(self, trending: Any):
        return await self._aask(TRENDING_ANALYSIS_PROMPT.format(trending=trending))



In [6]:
pip show metagpt

Name: metagpt
Version: 0.8.0
Summary: The Multi-Agent Framework
Home-page: https://github.com/geekan/MetaGPT
Author: Alexander Wu
Author-email: alexanderwu@deepwisdom.ai
License: MIT
Location: d:\ProgramData\Anaconda3\envs\wow-agent\Lib\site-packages
Requires: aiofiles, aiohttp, aioredis, anthropic, anytree, beautifulsoup4, channels, dashscope, faiss-cpu, fire, gitignore-parser, gitpython, google-generativeai, imap-tools, ipykernel, ipython, ipywidgets, jieba, lancedb, libcst, loguru, meilisearch, nbclient, nbformat, networkx, numpy, openai, openpyxl, pandas, Pillow, playwright, pydantic, python-docx, PyYAML, qdrant-client, qianfan, rank-bm25, rich, scikit-learn, semantic-kernel, setuptools, socksio, ta, tenacity, tiktoken, tqdm, typer, typing-extensions, typing-inspect, websocket-client, websockets, wrapt, zhipuai
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Role实现
# 对于V0.7 以上的版本，需要把老版本的
# self._init_actions 改为self.set_actions
class OssWatcher(Role):
    def __init__(
        self,
        name="Codey",
        profile="OssWatcher",
        goal="Generate an insightful GitHub Trending analysis report.",
        constraints="Only analyze based on the provided GitHub Trending data.",
    ):
        super().__init__(name=name, profile=profile, goal=goal, constraints=constraints)
        self.set_actions([CrawlOSSTrending, AnalysisOSSTrending])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        # By choosing the Action by order under the hood
        # todo will be first SimpleWriteCode() then SimpleRunCode()
        todo = self.rc.todo

        msg = self.get_memories(k=1)[0]  # find the most k recent messages
        result = await todo.run(msg.content)

        msg = Message(content=str(result), role=self.profile, cause_by=type(todo))
        self.rc.memory.add(msg)
        return msg

In [11]:
async def wxpusher_callback(msg: Message):
    print(msg.content)


async def trigger():
    # 这里设置了只触发五次，也可以用while True 永远执行下去
    for i in range(5):
        yield Message("https://github.com/trending")
        await asyncio.sleep(5)
        #  每隔五秒钟执行一次。
        # 也可以设置为每隔3600 * 24 秒执行一次
    

In [12]:
# 运行入口，
async def main():
    callbacks = []
    if not callbacks:
        async def _print(msg: Message):
            print(msg.content)
        callbacks.append(_print)

    # callback
    async def callback(msg):
        await asyncio.gather(*(call(msg) for call in callbacks))

    runner = SubscriptionRunner()
    await runner.subscribe(OssWatcher(), trigger(), callback)
    await runner.run()

In [13]:
await main()

2025-01-23 20:26:15.621 | INFO     | __main__:_act:17 - Codey(OssWatcher): ready to CrawlOSSTrending
2025-01-23 20:26:18.363 | INFO     | __main__:_act:17 - Codey(OssWatcher): ready to AnalysisOSSTrending


# Today's GitHub Trending: Exploring the Hottest Projects in AI, Emulation, and More!

## Today's Trends
Today, Python and C++ continue to dominate as the most popular programming languages. Key areas of interest include AI, emulation, and no-code solutions. The top popular projects are `shadPS4`, a PlayStation 4 emulator, and `exo`, a tool for running AI clusters at home.

## The Trends Categories
1. **Generative AI**
    - [exo](https://github.com/exo-explore/exo): Run your own AI cluster at home with everyday devices. Written in Python, it has gained 492 stars today.
    - [microsoft/generative-ai-for-beginners](https://github.com/microsoft/generative-ai-for-beginners): A beginner-friendly guide to building with generative AI, featuring 21 lessons. It has gained 251 stars today.
2. **Emulation**
    - [shadPS4](https://github.com/shadps4-emu/shadPS4): A PlayStation 4 emulator for Windows, Linux, and macOS, written in C++. It has gained 649 stars today.
3. **No-Code Solutions**
    -

2025-01-23 20:26:30.183 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.


# Today's GitHub Trending: Exploring the Hottest Projects in AI, Emulation, and More!

## Today's Trends
Today, Python and C++ continue to dominate as the most popular programming languages. Key areas of interest include AI, emulation, and no-code solutions. The top popular projects are `shadPS4`, a PlayStation 4 emulator, and `exo`, a tool for running AI clusters at home.

## The Trends Categories
1. **Generative AI**
    - [exo](https://github.com/exo-explore/exo): Run your own AI cluster at home with everyday devices. Written in Python, it has gained 492 stars today.
    - [microsoft/generative-ai-for-beginners](https://github.com/microsoft/generative-ai-for-beginners): A beginner-friendly guide to building with generative AI, featuring 21 lessons. It has gained 251 stars today.
2. **Emulation**
    - [shadPS4](https://github.com/shadps4-emu/shadPS4): A PlayStation 4 emulator for Windows, Linux, and macOS, written in C++. It has gained 649 stars today.
3. **No-Code Solutions**
    -

2025-01-23 20:26:35.191 | INFO     | __main__:_act:17 - Codey(OssWatcher): ready to CrawlOSSTrending
2025-01-23 20:26:36.037 | INFO     | __main__:_act:17 - Codey(OssWatcher): ready to AnalysisOSSTrending


...

2025-01-23 20:27:27.298 | INFO     | __main__:_act:17 - Codey(OssWatcher): ready to CrawlOSSTrending
2025-01-23 20:27:28.661 | INFO     | __main__:_act:17 - Codey(OssWatcher): ready to AnalysisOSSTrending


# Today's GitHub Trending: Exploring the Hottest Projects in AI, Emulation, and More!

## Today's Trends
Today, Python and C++ continue to dominate as the most popular programming languages. Key areas of interest include AI, emulation, and no-code solutions. The top popular projects are `shadPS4`, a PlayStation 4 emulator, and `exo`, a tool for running AI clusters at home.

## The Trends Categories
1. **Generative AI**
    - [exo](https://github.com/exo-explore/exo): Run your own AI cluster at home with everyday devices. Written in Python, it has gained 492 stars today.
    - [microsoft/generative-ai-for-beginners](https://github.com/microsoft/generative-ai-for-beginners): A beginner-friendly guide to building with generative AI, featuring 21 lessons. It has gained 251 stars today.
2. **Emulation**
    - [shadPS4](https://github.com/shadps4-emu/shadPS4): A PlayStation 4 emulator for Windows, Linux, and macOS, written in C++. It has gained 649 stars today.
3. **No-Code Solutions**
    -

2025-01-23 20:27:40.284 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.


 perfect for feeding into LLMs and other AI tools.
# Today's GitHub Trending: Exploring the Hottest Projects in AI, Emulation, and More!

## Today's Trends
Today, Python and C++ continue to dominate as the most popular programming languages. Key areas of interest include AI, emulation, and no-code solutions. The top popular projects are `shadPS4`, a PlayStation 4 emulator, and `exo`, a tool for running AI clusters at home.

## The Trends Categories
1. **Generative AI**
    - [exo](https://github.com/exo-explore/exo): Run your own AI cluster at home with everyday devices. Written in Python, it has gained 492 stars today.
    - [microsoft/generative-ai-for-beginners](https://github.com/microsoft/generative-ai-for-beginners): A beginner-friendly guide to building with generative AI, featuring 21 lessons. It has gained 251 stars today.
2. **Emulation**
    - [shadPS4](https://github.com/shadps4-emu/shadPS4): A PlayStation 4 emulator for Windows, Linux, and macOS, written in C++. It has gai

2025-01-23 20:27:45.909 | WARNING  | __main__:run:80 - Task Subscription-private_context=None private_config=None private_llm=<metagpt.provider.openai_api.OpenAILLM object at 0x00000165B7193210> name='Codey' profile='OssWatcher' goal='Generate an insightful GitHub Trending analysis report.' constraints='Only analyze based on the provided GitHub Trending data.' desc='' is_human=False role_id='' states=["0. <class '__main__.CrawlOSSTrending'>", "1. <class '__main__.AnalysisOSSTrending'>"] actions=[CrawlOSSTrending, AnalysisOSSTrending] rc=RoleContext(env=None, msg_buffer=MessageQueue(), memory=Memory(storage=[], index=defaultdict(<class 'list'>, {}), ignore_id=False), working_memory=Memory(storage=[], index=defaultdict(<class 'list'>, {}), ignore_id=False), state=-1, todo=None, watch={'metagpt.actions.add_requirement.UserRequirement'}, news=[], react_mode='by_order', max_react_loop=1) addresses={'Codey', '__main__.OssWatcher'} planner=Planner(plan=Plan(goal='', context='', tasks=[], task